In [1]:
!pip install synthcity
!pip install opacus==1.4.0

  Using cached opacus-1.5.4-py3-none-any.whl.metadata (8.7 kB)
Using cached opacus-1.5.4-py3-none-any.whl (254 kB)
  Using cached opacus-1.4.0-py3-none-any.whl.metadata (7.9 kB)
Using cached opacus-1.4.0-py3-none-any.whl (224 kB)


In [2]:
!unrar x data.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from data.rar

Creating    data                                                      OK
Extracting  data/california_housing.csv                                   24%  OK 
Extracting  data/insurance.csv                                            31%  OK 
Extracting  data/my_classification.csv                                    55%  OK 
Extracting  data/my_regression.csv                                        80%  OK 
Extracting  data/titanic.csv                                              89%  OK 
Extracting  data/two_moons.csv                                            99%  OK 
All OK


In [3]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, rand, hp, Trials, STATUS_OK
from hyperopt import space_eval
from synthcity.plugins import Plugins
from synthcity.plugins.generic.plugin_ddpm import TabDDPMPlugin as DDPM
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import tqdm
from sklearn.metrics import f1_score, r2_score
from xgboost import XGBClassifier, XGBRegressor

[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [4]:
dir_datasets = 'data/'

# Загрузка реальных датасетов
real_data_1 = pd.read_csv(dir_datasets+'insurance.csv')
real_data_2 = pd.read_csv(dir_datasets+'my_classification.csv')
real_data_3 = pd.read_csv(dir_datasets+'my_regression.csv')
real_data_4 = pd.read_csv(dir_datasets+'two_moons.csv')
real_data_5 = pd.read_csv(dir_datasets+'california_housing.csv')
real_data_6 = pd.read_csv(dir_datasets+'titanic.csv')

# Словарь датасетов для удобства
datasets = {
        'insurance': {
                        "data": real_data_1,
                        "task": "regression",
                        "target": "expenses",
                        "num_columns":
                        ["age", "bmi", "children"],
                        "cat_columns":
                        ["sex", "smoker", "region"]
                    },

}

for name, data in datasets.items():
    print(f" Информация о датасете {name}:")
    print(f" Количество строк: {data['data'].shape[0]}")
    print(f" Количество колонок: {data['data'].shape[1]}")
    print(f" Колонки: {list(data['data'].columns)}")
    print(f" Задача: {data['task']}")
    print(f" Целевая переменная: {data['target']}")
    print(f" Числовые признаки: {data['num_columns']}")
    print(f" Категориальные признаки: {data['cat_columns']}")
    print()

 Информация о датасете insurance:
 Количество строк: 1338
 Количество колонок: 7
 Колонки: ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses']
 Задача: regression
 Целевая переменная: expenses
 Числовые признаки: ['age', 'bmi', 'children']
 Категориальные признаки: ['sex', 'smoker', 'region']



In [5]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [6]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [7]:
# Определение пространства поиска гиперпараметров для DDPM
ddpm_space = {
    'batch_size': hp.choice('batch_size', [4096]),
    'lr': hp.qloguniform('lr', np.log(3.5e-4), np.log(9.2e-4), 1e-5),
    'num_timesteps': hp.choice('num_timesteps', [1000]),
    'n_layers_hidden': hp.choice('n_layers_hidden', [2, 4, 6]),
    'n_units_hidden': hp.choice('n_units_hidden', [256, 512, 1024]),
    'transformation_num_type': hp.choice('transformation_num_type', ['None']),
    'transformation_cat_type': hp.choice('transformation_cat_type', ['None'])
}

In [8]:
def evaluate_pair(real_data, synthetic_data, cat_columns):
    # One-hot энкодинг для категориальных признаков
    real_data_encoded = pd.get_dummies(real_data, columns=cat_columns, drop_first=True)

    if hasattr(synthetic_data, "dataframe"):
        synthetic = synthetic_data.dataframe()
    else:
        synthetic = synthetic_data

    synthetic_encoded = pd.get_dummies(synthetic, columns=cat_columns, drop_first=True)

    # После get_dummies
    real_data_encoded = real_data_encoded.astype(float)
    synthetic_encoded = synthetic_encoded.astype(float)

    # Корреляции
    corr_real = real_data_encoded.corr()
    corr_synth = synthetic_encoded.corr()

    # Абсолютное отклонение корреляций
    pairwise_diff = np.abs(corr_real - corr_synth)

    # Усреднённое отклонение
    mean_diff = pairwise_diff.mean().mean()

    # Проверяем на NaN
    if np.isnan(mean_diff):
        print("Warning: mean_diff is NaN, returning 0")
        return 0.0

    return 1 - mean_diff  # ближе к 1 — лучше

In [9]:
import sys
import os
import builtins
import contextlib

@contextlib.contextmanager
def suppress_all_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        old_print = builtins.print
        builtins.print = lambda *args, **kwargs: None
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr
            builtins.print = old_print

In [10]:
from sklearn.model_selection import KFold

def generate_and_evaluate_pair_scores(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    columns_dataset = dataset_info['num_columns'] + dataset_info['cat_columns'] + [dataset_info['target']]
    data = process_data(dataset_info['data'][columns_dataset], dataset_info['num_columns'], dataset_info['cat_columns'],
                        params['transformation_num_type'], params['transformation_cat_type'])

    kf = KFold(n_splits=n_splits, shuffle=False)
    pair_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        # Создание и обучение DDPM с заданными параметрами
        ddpm = DDPM(
            lr=params['lr'],
            num_timesteps=params['num_timesteps'],
            model_params = dict(n_layers_hidden=params['n_layers_hidden'],
                                n_units_hidden=params['n_units_hidden'],
                                dropout=0.0),
            batch_size=params['batch_size'],
            #n_iter=params['n_iter']
        )

        # Обучение модели на реальных данных
        with suppress_all_output():
            ddpm.fit(train_data)

        # Генерация синтетических данных
        synthetic_data = ddpm.generate(len(train_data))

        cat_columns = dataset_info["cat_columns"].copy()
        if dataset_info["task"] == 'classification':
            cat_columns.append(dataset_info['target'])

        # Вычисляем Pair-score
        score = evaluate_pair(
            test_data,
            synthetic_data,
            cat_columns
        )
        pair_scores.append(score)
    print()
    return np.mean(pair_scores)

In [11]:
def optimize_dataset(dataset_name, dataset_info, max_evals=30):
    print(f"Оптимизация для датасета: {dataset_name}")

    # Создаем объект для хранения истории поиска
    trials = Trials()

    # Определяем целевую функцию для оптимизации
    def objective(params):
        print("="*50)
        pair_score = generate_and_evaluate_pair_scores(params, dataset_info)

        print(f"Pair-score: {pair_score}")
        print(params)
        print("="*50)

        # Для ML-Efficacy мы хотим максимизировать метрику, поэтому возвращаем отрицательное значение
        return {
            'loss': -pair_score,  # отрицательное для максимизации
            'status': STATUS_OK,
            'pair_score': pair_score
        }

    # Запускаем оптимизацию с помощью TPE алгоритма
    rng = np.random.default_rng(42)  # Используем фиксированный seed
    best = fmin(
        fn=objective,
        space=ddpm_space,
        algo=rand.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=rng
    )

    # Автоматическое декодирование параметров
    best_params = space_eval(ddpm_space, best)

    # Находим лучший результат
    best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
    best_pair_score = best_trial['result'].get('pair_score', None)

    results = {
        'best_params': best_params,
        'best_pair_score': best_pair_score,
        'pair_score_diff_from_optimal': -best_trial['result']['loss']  # убираем минус для читаемости
    }

    print(f"Лучший Pair-score для {dataset_name}: {best_pair_score}")
    print(f"Лучшие параметры: {best_params}")
    print('-' * 50)

    return results

In [ ]:
# Словарь для хранения результатов
all_results = {}

# Указываем количество итераций для каждого датасета
max_evals = 50

# Запускаем оптимизацию для каждого датасета
for dataset_name in datasets:
    data = datasets[dataset_name]
    all_results[dataset_name] = optimize_dataset(dataset_name, data, max_evals)

Оптимизация для датасета: insurance
Pair-score: 0.931744387879803
{'batch_size': 4096, 'lr': 0.00064, 'n_layers_hidden': 6, 'n_units_hidden': 256, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
Pair-score: 0.7949896579396984
{'batch_size': 4096, 'lr': 0.00063, 'n_layers_hidden': 2, 'n_units_hidden': 1024, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
Pair-score: 0.7606910941183366
{'batch_size': 4096, 'lr': 0.00038, 'n_layers_hidden': 2, 'n_units_hidden': 256, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
Pair-score: 0.9111516243048974
{'batch_size': 4096, 'lr': 0.00043000000000000004, 'n_layers_hidden': 4, 'n_units_hidden': 512, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
Pair-score: 0.8780760949047174
{'batch_size': 4096, 'lr': 0.00038, 'n_layers_hidden': 4, 'n_units_hidden': 256, 'num_timesteps': 

In [ ]:
for dataset_name in datasets:
    print(dataset_name, all_results[dataset_name]['best_pair_score'])